In [ ]:
import os
import numpy as np
import torch

from nuscenes.nuscenes import NuScenes

In [ ]:
DIR_DATA = "/ssd/Datasets_and_code/nuscenes_depth_estimation/dataset/dataset_radar_cam"
DIR_NUSCENES = "/ssd/Datasets_and_code/nuscenes_depth_estimation/dataset/nuscenes_mini"
VERSION = "v1.0-mini"

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

In [ ]:
if not os.path.exists(DIR_DATA):
    os.makedirs(DIR_DATA)

In [ ]:
nusc = NuScenes(version=VERSION, dataroot = DIR_NUSCENES, verbose=False)

In [ ]:
# Only use clear day scenes
nuscenes_scenes = []

for scene in nusc.scene:         
    nuscenes_scenes.append(scene['token'])
    
np.random.shuffle(nuscenes_scenes)    
    
print(len(nusc.scene), 'total scenes')

In [ ]:
# Get train, val and test scenes
n_train_scenes = int(round(len(nuscenes_scenes) * train_ratio))
n_val_scenes = int(round(len(nuscenes_scenes) * val_ratio))
n_test_scenes = len(nuscenes_scenes) - n_train_scenes - n_val_scenes

train_scenes = []
val_scenes = []
test_scenes = []

train_scenes += nuscenes_scenes[:n_train_scenes]
val_scenes += nuscenes_scenes[ n_train_scenes: n_train_scenes + n_val_scenes ]
test_scenes += nuscenes_scenes[n_train_scenes + n_val_scenes : ]

In [ ]:
train_sample_idx = []
val_sample_idx = []
test_sample_idx = []
idx_to_token = {}


for idx, sample in enumerate(nusc.sample):            
    if sample['scene_token'] in train_scenes:
        train_sample_idx.append(idx)
    elif sample['scene_token'] in val_scenes:
        val_sample_idx.append(idx)
    elif sample['scene_token'] in test_scenes:
        test_sample_idx.append(idx)
    idx_to_token[idx] = sample['token']

print(len(train_sample_idx), len(val_sample_idx), len(test_sample_idx))


all_idx = train_sample_idx + val_sample_idx + test_sample_idx
#print("train sample idx: " + str(train_sample_idx))
#print("val sample idx: " + str(val_sample_idx))
#print("test sample idx: " + str(test_sample_idx))
#print(idx_to_token)

In [ ]:
# Save in dictionary
data_split = {'all_indices': all_idx,
                  'train_sample_indices': train_sample_idx,
                  'val_sample_indices': val_sample_idx,
                  'test_sample_indices':  test_sample_idx }


torch.save(data_split, os.path.join(DIR_DATA, 'data_split.tar'))
torch.save(idx_to_token, os.path.join(DIR_DATA, 'idx_to_token.tar'))